In [47]:
import numpy as np
import matplotlib.pyplot as mpl
from sklearn.metrics import mean_squared_error

In [48]:
d = 10
n_train = 10000
n_test = 100

x_train = np.random.randn(n_train, d)
x_test = np.random.randn(n_test, d)
mean_train = np.empty(d)
norm_tab = np.empty(n_train)

norm_train = np.sum(x_train * x_train, axis = 0) ** 0.5
x_train = (x_train / norm_train) * (d ** 0.5) 
    
norm_test = np.sum(x_test * x_test, axis = 0) **0.5
x_test = (x_test / norm_test) * (d**0.5)

#On vérifie que notre data set soit uniforme
mean_train = np.sum(x_train, axis = 1) / n
    
#On vérifie que notre dataset ait une norme de sqrt(d)
norm_tab = np.sum(x_train * x_train, axis = 0) ** 0.5
    
print(mean_train)
print(norm_tab)

[ 4.76033058e-06  1.30472388e-05  1.64211717e-05 ...  4.26304984e-06
 -2.82637457e-05  2.12524363e-06]
[3.16227766 3.16227766 3.16227766 3.16227766 3.16227766 3.16227766
 3.16227766 3.16227766 3.16227766 3.16227766]


In [49]:
tau = 0.3 
noise_level = tau * tau

#On veut E(noise) == 0 et E(noise^2) == tau^2
noise_train = np.random.randn(n_train, 1) * noise_level
noise_test = np.random.randn(n_test, 1) * noise_level

print(noise_train)

[[ 0.01805749]
 [-0.06229601]
 [ 0.06265033]
 ...
 [-0.02420928]
 [ 0.11474633]
 [-0.01064783]]


In [50]:
F1 = 1

sample_params = np.random.randn(d, 1)
    
norm = np.sum(sample_params * sample_params) #should be equal to F1^2
sample_params = sample_params * (F1 / (norm**0.5))

print(sample_params)
print(np.sum(sample_params * sample_params) ** 0.5)

[[-0.63042882]
 [ 0.02091969]
 [-0.25413631]
 [ 0.18569001]
 [ 0.31498142]
 [ 0.14747567]
 [-0.52809966]
 [-0.00218809]
 [-0.32123861]
 [ 0.00227512]]
0.9999999999999999


In [51]:
y_train = x_train @ sample_params + noise_train
y_test = x_test @ sample_params + noise_test
print(sample_y)

[[-0.50890513]
 [-2.04905717]
 [-0.22448405]
 ...
 [ 0.27923782]
 [ 0.28757629]
 [-0.38532663]]


In [52]:
#Quelques fonctions d'activation :

class Sigmoid :
    @staticmethod
    def function(x):
        return 1/(1+np.exp(-x))
    
    @staticmethod
    def gradient(x):
        s = Sigmoid.function(x)
        return s * (1- s)
    
class Tanh :
    @staticmethod
    def function(x):
        return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    
    @staticmethod
    def gradient(x):
        t = Tanh(x)
        return 1-t**2
    
class Relu :
    @staticmethod
    def function(x):
        return np.maximum(0, x)
    
    @staticmethod
    def gradient(x):
        return np.maximum(0, x) / x


In [53]:
#Quelques Loss Functions :
class MSE:
    @staticmethod
    def loss(y_real, y_hat):
        return (1/(y_real.shape[0] * y_real.shape[1])) * np.sum(np.sum((y_hat - y_real)**2, axis = 0))
    
    @staticmethod
    def gradient(y_real, y_hat):
        return (2/(y_real.shape[0] * y_real.shape[1])) * (y_hat - y_real)

In [58]:
#L'architecture

class Network:
    def __init__(self, dimension_hidden, activation1, activation2):
        """
        dimension_hidden est le nombre de paramètres dans le hidden layer (N dans le papier de Mei et Montanari)
        activation1 est la fonction d'activation du hidden layer
        activation2 est la fonction d'activation de l'output layer
        """
        
        self.nb_layers = 3 #input, hidden, output
        self.dimensions = (d, dimension_hidden, 1)
        
        self.loss_tab = None
        
        self.learning_rate = {}
        self.learning_rate[1] = None;  #learning rate du hidden layer
        self.learning_rate[2] = None;  #learning rate du output layer
        
        self.weights = {}
        self.bias = {}
        
        #on initialise les weights et les bias aléatoirement
        for i in range(self.nb_layers -1):
            self.weights[i + 1] = np.random.randn(self.dimensions[i], self.dimensions[i + 1]) / np.sqrt(self.dimensions[i])
            self.bias[i + 1] = np.zeros(self.dimensions[i + 1])
            
        self.activations = {}
        self.activations[2] = activation1
        self.activations[3] = activation2
        
    def forward_pass(self, x):
        """
        x est un vecteur de notre data
        """
        z = {}
        a = {1:x} #l'input layer n'a pas d'activation function, a[1] est donc égal à x
        
        for i in range(1, self.nb_layers):
            z[i + 1] = a[i] @ self.weights[i] + self.bias[i] #Z = XW + b
            a[i + 1] = self.activations[i + 1].function(z[i + 1]);
            
        return z, a
    
    def predict(self, x):
        z, a = self.forward_pass(x)
        return a[self.nb_layers]
    
    def back_propagation(self, z, a, real_value):
        y_hat = a[self.nb_layers]
        
        #On calcule delta et la dérivée partielle à l'output layer
        delta = self.loss_function.gradient(real_value, y_hat) * self.activations[self.nb_layers].gradient(y_hat)
        partial_deriv = a[self.nb_layers - 1].T @ delta
        
        update_parameters = {
            self.nb_layers - 1: (partial_deriv, delta)
        }
        
        for i in reversed(range(2, self.nb_layers)):
            delta = (delta @ self.weights[i].T) * self.activations[i].gradient(z[i])
            partial_deriv = a[i - 1].T @ delta 
            update_parameters[i - 1] = (partial_deriv, delta)
            
        for k, v in update_parameters.items():
            self.update_weights_and_bias(k, v[0], v[1])
            
    def update_weights_and_bias(self, index, partial_deriv, delta):
        self.weights[index] -= self.learning_rate[index] * partial_deriv
        self.bias[index] -= self.learning_rate[index] * np.mean(delta, 0)
        
    def fit(self, x, y_real, loss, nb_iterations, batch_size, learning_rate1, learning_rate2):
        #On vérifie qu'on a autant de x que de y
        if not (x.shape[0] == y_real.shape[0]):
            raise Exception
            
        self.loss_tab = np.empty((nb_iterations, x.shape[0] // batch_size))
        
        self.loss_function = loss
        self.learning_rate[1] = learning_rate1
        self.learning_rate[2] = learning_rate2
        
        #We use batch gradient descent
        for i in range(nb_iterations):
            for j in range(x.shape[0] // batch_size):
                start = j * batch_size
                end = (j + 1) * batch_size
                z, a = self.forward_pass(x[start:end])
                self.loss_tab[i][j] = self.loss_function.loss(y_real[start:end], a[self.nb_layers])
                self.back_propagation(z, a, y_real[start:end])
            if(i % 10) == 0:
                print("Loss at Iteration {} is {}".format(i, self.loss_tab[i]))
        

In [59]:
nn = Network(10000, Relu, Sigmoid)
nn.fit(x_train, y_train, MSE, 101, 1000, 0, 0.03)

z, a = nn.forward_pass(x_test)
print(nn.loss_function.loss(y_test, a[nn.nb_layers]))

Loss at Iteration 0 is [0.26552766 0.25932027 0.25860809 0.25182894 0.24571689 0.24615723
 0.24564879 0.24334162 0.24637182 0.23718969]
Loss at Iteration 10 is [0.1000337  0.09896867 0.09802594 0.09522092 0.09394852 0.09370557
 0.09398595 0.09355521 0.09704572 0.09218913]
Loss at Iteration 20 is [0.05170545 0.05171131 0.05100437 0.04992534 0.04931549 0.04916219
 0.04963825 0.04955048 0.05263307 0.04916221]
Loss at Iteration 30 is [0.03360323 0.03382852 0.03326525 0.03299304 0.03239684 0.03231616
 0.03284391 0.03283458 0.03546886 0.03259338]
Loss at Iteration 40 is [0.02508122 0.02534266 0.02486698 0.0250338  0.02437728 0.0243341
 0.02487216 0.02489537 0.02717507 0.02463399]
Loss at Iteration 50 is [0.02040193 0.02065293 0.02023561 0.02066846 0.01995979 0.01993453
 0.02046732 0.02051207 0.02252125 0.02019877]
Loss at Iteration 60 is [0.01754769 0.01777621 0.01740059 0.01800728 0.01726257 0.01724488
 0.01776608 0.01782837 0.01962872 0.01746237]
Loss at Iteration 70 is [0.01567077 0.01587